# **ML.NET 的一些例子** #

In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.7.0

In [ ]:
using System;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
public class HouseData
{
    public float Size { get; set; }
    public float Price { get; set; }
}
   
public class Prediction
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

In [ ]:
MLContext mlContext = new MLContext();

In [ ]:
HouseData[] houseData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.4F, Price = 3.7F } };
IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

In [ ]:
var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Size" }).Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", maximumNumberOfIterations: 100));

In [ ]:
var model = pipeline.Fit(trainingData);

In [ ]:
var size = new HouseData() { Size = 2.5F };
var price = mlContext.Model.CreatePredictionEngine<HouseData, Prediction>(model).Predict(size);

In [ ]:
Console.WriteLine($"Predicted price for size: {size.Size*1000} sq ft= {price.Price*100:C}k");

Predicted price for size: 2500 sq ft= ¤276.03k


In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:

public class SentimentIssue
{
  [LoadColumn(0)]
  public string Text { get; set; }
  [LoadColumn(1)]
  public bool Label { get; set; }
}

In [ ]:
public class SentimentPrediction
{
   [ColumnName("PredictedLabel")]
   public bool Prediction { get; set; }
   
   public float Probability { get; set; }

   public float Score { get; set; }
}

In [ ]:
var mlContext = new MLContext(seed: 1);

In [ ]:
IDataView trainingData = mlContext.Data.LoadFromTextFile<SentimentIssue>(@"datasets/mlnet/train_data.tsv", hasHeader: true);
IDataView testData = mlContext.Data.LoadFromTextFile<SentimentIssue>(@"datasets/mlnet/train_data.tsv", hasHeader: true);

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("Features", nameof(SentimentIssue.Text));

In [ ]:
var trainer = mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression("Label", "Features");
var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
ITransformer trainedModel = trainingPipeline.Fit(trainingData);

In [ ]:
var predictions = trainedModel.Transform(testData);
var metrics = mlContext.BinaryClassification.Evaluate(data: predictions, labelColumnName: "Label", scoreColumnName: "Score");

In [ ]:
Console.WriteLine($"*       Accuracy: {metrics.Accuracy:P2}");
Console.WriteLine($"*       Area Under Curve:      {metrics.AreaUnderRocCurve:P2}");
Console.WriteLine($"*       Area under Precision recall Curve:  {metrics.AreaUnderPrecisionRecallCurve:P2}");
Console.WriteLine($"*       F1Score:  {metrics.F1Score:P2}");
Console.WriteLine($"*       LogLoss:  {metrics.LogLoss:#.##}");
Console.WriteLine($"*       LogLossReduction:  {metrics.LogLossReduction:#.##}");
Console.WriteLine($"*       PositivePrecision:  {metrics.PositivePrecision:#.##}");
Console.WriteLine($"*       PositiveRecall:  {metrics.PositiveRecall:#.##}");
Console.WriteLine($"*       NegativePrecision:  {metrics.NegativePrecision:#.##}");
Console.WriteLine($"*       NegativeRecall:  {metrics.NegativeRecall:P2}");

*       Accuracy: 70.80 %
*       Area Under Curve:      79.01 %
*       Area under Precision recall Curve:  78.55 %
*       F1Score:  70.86 %
*       LogLoss:  .89
*       LogLossReduction:  .11
*       PositivePrecision:  .71
*       PositiveRecall:  .71
*       NegativePrecision:  .71
*       NegativeRecall:  70.60 %


In [ ]:
mlContext.Model.Save(trainedModel, trainingData.Schema, @"datasets/mlnet/SentimentModel.zip");

In [ ]:
SentimentIssue sampleStatement = new SentimentIssue { Text = "This is a very good film" };

In [ ]:
var predEngine = mlContext.Model.CreatePredictionEngine<SentimentIssue, SentimentPrediction>(trainedModel);

In [ ]:
var resultprediction = predEngine.Predict(sampleStatement);

In [ ]:
Console.WriteLine($"=============== Single Prediction  ===============");
Console.WriteLine($"Text: {sampleStatement.Text} | Prediction: {(Convert.ToBoolean(resultprediction.Prediction) ? "Toxic" : "Non Toxic")} sentiment | Probability of being toxic: {resultprediction.Probability} ");

=============== Single Prediction  ===============
Text: This is a very good film | Prediction: Toxic sentiment | Probability of being toxic: 0.6393101 
